## 데이터 다운로드
--- 
아래 셀을 실행시켜 데이터를 colab에 불러옵니다.  
셀 실행은 <br>Windows: `Ctrl` + `Enter`<br>
Mac: `Shift` + `Enter` <br>
를 이용해 실행 시킬 수 있습니다. 

In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3l6g8j3'

import zipfile
with zipfile.ZipFile('3l6g8j3', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-02-06 15:07:42--  https://bit.ly/3l6g8j3
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://docs.google.com/uc?export=download&id=1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu [following]
--2022-02-06 15:07:42--  https://docs.google.com/uc?export=download&id=1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu
Resolving docs.google.com (docs.google.com)... 142.250.97.138, 142.250.97.113, 142.250.97.101, ...
Connecting to docs.google.com (docs.google.com)|142.250.97.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/macg0upp6f7lo76ta23h7gv4oie0ijlo/1644160050000/17946651057176172524/*/1n-cTWLp385P-DmmNuMAg6zJDw8MNAPwu?e=download [following]
--2022-02-06 15:07:45--  https://doc-0g-10-docs.googleusercontent.com/docs/securesc/ha0ro9

In [2]:
# 데이터 불러오기
import pandas as pd

train = pd.read_csv('data/train.csv',usecols = ['category','data'])
test = pd.read_csv('data/test.csv',usecols = ['data'])

In [3]:
# train,test 데이터에서  '\\n' 단어 제거

train['data'] = train['data'].apply(lambda x : str(x).replace('\\n',' ')) # 텍스트 데이터에서 '\\n' 문자열을 ' '로 변경
test['data'] = test['data'].apply(lambda x : str(x).replace('\\n',' ')) # 텍스트 데이터에서 '\\n' 문자열을 ' '로 변경

In [4]:
# 라이브러리 로딩
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF으로 train 데이터를 피처 벡터화 변환 수행

vect = TfidfVectorizer()
vect.fit(train['data'])
train_x =  vect.transform(train['data'])

print('train data size', train_x.shape)

train data size (40000, 697226)


In [5]:
from sklearn.model_selection import train_test_split
#train_test_split() 메소드를 이용해 train/validation 데이터 나누기 
x_train,x_valid, y_train, y_valid = train_test_split(train_x,train['category'],stratify = train['category'])

## 실습
---


In [6]:
# LightGBM을 이용해 학습 및 검증 진행
from lightgbm import LGBMRegressor

#모델 정의 
model = LGBMRegressor()

#평가 산식을 AUC로 설정, n_estimators 기준 5번 마다 결과값 출력하게 모델 학습.
model.fit(train_x, train['category'],
          eval_set=[ (x_train, y_train), (x_valid, y_valid) ],
          eval_metric='auc', verbose=5)

[5]	valid_0's auc: 0.748618	valid_0's l2: 0.521822	valid_1's auc: 0.746711	valid_1's l2: 0.520125
[10]	valid_0's auc: 0.777249	valid_0's l2: 0.455408	valid_1's auc: 0.77662	valid_1's l2: 0.452152
[15]	valid_0's auc: 0.7942	valid_0's l2: 0.416148	valid_1's auc: 0.795026	valid_1's l2: 0.413184
[20]	valid_0's auc: 0.807472	valid_0's l2: 0.391471	valid_1's auc: 0.809959	valid_1's l2: 0.387826
[25]	valid_0's auc: 0.819458	valid_0's l2: 0.374663	valid_1's auc: 0.81984	valid_1's l2: 0.370974
[30]	valid_0's auc: 0.830091	valid_0's l2: 0.361912	valid_1's auc: 0.8308	valid_1's l2: 0.358228
[35]	valid_0's auc: 0.840576	valid_0's l2: 0.352122	valid_1's auc: 0.841772	valid_1's l2: 0.347999
[40]	valid_0's auc: 0.848646	valid_0's l2: 0.343905	valid_1's auc: 0.847935	valid_1's l2: 0.339922
[45]	valid_0's auc: 0.853207	valid_0's l2: 0.337016	valid_1's auc: 0.852349	valid_1's l2: 0.333306
[50]	valid_0's auc: 0.857342	valid_0's l2: 0.331056	valid_1's auc: 0.856211	valid_1's l2: 0.327363
[55]	valid_0's au

LGBMRegressor()